In [ ]:
import h5py

# Specify the full path to the HDF5 file including the folder
hdf5_filename = '/mnt/e/PdM/DataX/NASA/Temp_Data/processed_data.h5'

# Open the HDF5 file for reading
hdf5_file = h5py.File(hdf5_filename, 'r')

# Load the datasets into variables
X = hdf5_file['X_train'][:]
X_test = hdf5_file['X_test'][:]
y = hdf5_file['y'][:]
y_test = hdf5_file['y_test'][:]

# Close the HDF5 file
hdf5_file.close()

In [ ]:
# Calculate the split index for 70% training and 20% validation
split_index = int(len(X) * 0.9)

# split_index = 4069786

# Split the data into training and validation sets
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Embedding
from tensorflow.keras.layers import MultiHeadAttention

# Define some constants for your model (these may need to be adjusted for your specific case)
n_features = X.shape[1] # Number of features in the input data
embedding_dim = 64 # Dimension of the embedding space
num_heads = 4 # Number of heads in the multi-head attention mechanism
dff = 256 # Dimension of the feed-forward network
num_layers = 4 # Number of Transformer encoder layers

# Positional encoding for adding information about sequence order
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
                                     i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
                                     d_model=d_model)
        # Apply sin to even indices in the array; cos to odd indices
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

# Transformer encoder layer
def transformer_encoder(inputs, embedding_dim, num_heads, dff, rate=0.1):
    # Multi-head attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(inputs, inputs)
    # Skip connection and layer normalization
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attention_output)
    # Feed forward network
    ffn_output = Dense(dff, activation='relu')(out1)
    ffn_output = Dense(embedding_dim)(ffn_output)
    # Skip connection and layer normalization
    out2 = LayerNormalization(epsilon=1e-6)(out1 + ffn_output)
    return out2

# Input layer
inputs = tf.keras.Input(shape=(None, n_features))
# Embedding and positional encoding
x = Dense(embedding_dim)(inputs)
x = PositionalEncoding(n_features, embedding_dim)(x)

# Transformer blocks
for _ in range(num_layers):
    x = transformer_encoder(x, embedding_dim, num_heads, dff)

# Output layer
x = Dense(50, activation='relu')(x) # Additional dense layer (optional)
x = Dense(1, activation='linear')(x) # Output layer

# Create the model
transformer_model = tf.keras.Model(inputs=inputs, outputs=x)

# Model summary
transformer_model.summary()


In [ ]:
# Compile the model
transformer_model.compile(optimizer=Adam(learning_rate=0.001, amsgrad=True), loss='mean_squared_error', 
                          metrics=['mean_squared_error'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Train the model
history = transformer_model.fit(X_train, y_train,
                                epochs=epochs,
                                batch_size=batch_size,
                                validation_data=(X_val, y_val),
                                callbacks=[early_stopping])
